### **Tutorial 8: Extracting and Transforming Data**

In this tutorial, we will walk through the process of **extracting** and **transforming** data from a SQL database using Python. We will:
- Connect to a SQL Server database using `SQLAlchemy` and `pyodbc`
- Extract data from the **Sales.Orders** table
- Transform the data to filter only **date-related columns**

---

#### **Step 1: Install Required Libraries**
Before starting, ensure you have the required Python libraries installed. If not, install them using:

```bash
pip install pandas sqlalchemy pyodbc
```

---

#### **Step 2: Import Required Libraries**
We begin by importing the necessary libraries:

```python
import pandas as pd
from sqlalchemy import create_engine
```

---

#### **Step 3: Define the Database Connection**
We use **SQLAlchemy** to create a connection string for a Microsoft SQL Server database.

```python
# Define the ODBC connection parameters
driver = 'ODBC Driver 18 for SQL Server' 

params = urllib.parse.quote_plus(
    f"DRIVER={{{driver}}};SERVER={server};DATABASE={database};"
    f"UID={username};PWD={password};ENCRYPT=yes;TrustServerCertificate=yes"
)

# Create the database engine
engine = create_engine(f"mssql+pyodbc:///?odbc_connect={params}")
```

Replace:
- **`your_server`** with the actual SQL Server name.
- **`your_database`** with your database name.
- **`your_username`** and **`your_password`** with your credentials.

---

In [54]:
import urllib
import pyodbc
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os
import pandas as pd


load_dotenv()  # Load environment variables from .env file

server = os.getenv("DB_SERVER")
database = os.getenv("DB_NAME")
username = os.getenv("DB_USER")
password = os.getenv("DB_PASSWORD") 
driver = 'ODBC Driver 18 for SQL Server' 

params = urllib.parse.quote_plus(
    f"DRIVER={{{driver}}};SERVER={server};DATABASE={database};"
    f"UID={username};PWD={password};ENCRYPT=yes;TrustServerCertificate=yes"
)

engine = create_engine(f"mssql+pyodbc:///?odbc_connect={params}")

query = """ 
    SELECT * FROM Sales.Orders;
    """

raw_data = pd.read_sql(query, engine)

#### **Step 4: Extract Data from SQL Database**
### **Explanation:**
- The `extract()` function executes an SQL query to retrieve all columns from the **Sales.Orders** table.
- `pd.read_sql()` fetches the query results into a **pandas DataFrame**.

---

In [55]:

def extract(engine):
    query = """ 
        SELECT * FROM Sales.Orders;
    """
    raw_data = pd.read_sql(query, engine)

    return raw_data
engine = create_engine(f"mssql+pyodbc:///?odbc_connect={params}")

raw_sales_data  = extract(engine)

## **Step 5: Transform the Data**
Define a function to filter the data, keeping only columns that contain **date-related** information.

```python
Example codes: 
# Extract Columns
no_null_columns = raw_data.columns[~raw_data.isnull().any()].to_list()
clean_data = raw_data[no_null_columns]
# record filtering
start_date = pd.to_datetime('2010-01-01').date()
clean_data = raw_data.loc[raw_data['OrderDate']>start_date, :]
```


In [56]:
# Complete the transform function to return the columns that includes the date data. 
def transform(raw_data):
  	# Edit the below codes to Filter rows and columns
    clean_data = raw_data.copy()
    return clean_data

clean_data = transform(raw_sales_data)

## **Expected Output**
If the `Sales.Orders` table has columns like:
| OrderID | OrderDate   | ExpectedDeliveryDate  | PickingCompletedWhen | LastEditedWhen |
|---------|------------|-----------|------------|-------------|
| 1       | 2025-01-01 | 2025-01-03 | C001       | 150.00      |
| 2       | 2025-02-15 | 2025-02-17 | C002       | 200.00      |


---

In [57]:
# Extract raw data from the database
raw_sales_data = extract(engine)

# Transform the raw data to include only date/time columns
clean_sales_data = transform(raw_sales_data)

# Display the transformed data
print(clean_sales_data.head())

   OrderID  CustomerID  SalespersonPersonID  PickedByPersonID  \
0        1         832                    2               NaN   
1        2         803                    8               NaN   
2        3         105                    7               NaN   
3        4          57                   16               3.0   
4        5         905                    3               NaN   

   ContactPersonID  BackorderOrderID   OrderDate ExpectedDeliveryDate  \
0             3032              45.0  2013-01-01           2013-01-02   
1             3003              46.0  2013-01-01           2013-01-02   
2             1209              47.0  2013-01-01           2013-01-02   
3             1113               NaN  2013-01-01           2013-01-02   
4             3105              48.0  2013-01-01           2013-01-02   

  CustomerPurchaseOrderNumber  IsUndersupplyBackordered Comments  \
0                       12126                      True     None   
1                       15342     

#### **Step 7: Load the Data**
After transforming the data, save it to a CSV file for further use.


In [58]:
# Define the output file path
output_path = "clean_sales_data.csv"

# Save the cleaned data to a CSV file
clean_sales_data.to_csv(output_path, index=False)

print(f"Data successfully saved to {output_path}")

Data successfully saved to clean_sales_data.csv


Apache **Parquet** is a columnar storage format that is optimized for big data processing. Unlike traditional row-based formats (such as CSV), Parquet stores data by columns, making it more efficient for analytical queries.

In [59]:
# Define the output file path
output_path = "clean_sales_data.parquet"

# Save the cleaned data to a CSV file
clean_sales_data.to_csv(output_path, index=False)

print(f"Data successfully saved to {output_path}")

Data successfully saved to clean_sales_data.parquet
